In [1]:
import pandas as pd
import sys
import numpy as np
sys.path.append('../src')
from functions import *
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import ttest_ind, chi2_contingency
from scipy.stats import ttest_ind

# Loading the Data:

In [2]:
df_clients_profile = pd.read_csv('../Data/Raw/df_final_demo.txt')
df_web_data_1 = pd.read_csv('../Data/Raw/df_final_web_data_pt_1.txt')
df_web_data_2 = pd.read_csv('../Data/Raw/df_final_web_data_pt_2.txt')
df_experiment_clients = pd.read_csv('../Data/Raw/df_final_experiment_clients.txt')
pd.set_option('display.max_columns', None) # To display all columns
df_web_data = pd.concat([df_web_data_1, df_web_data_2], ignore_index= True)

# Exploratory Data Analysis (EDA):

## Initial Exploration

In [3]:
df_clients_profile.head(10)

,client_id,clnt_tenure_yr,clnt_tenure_mnth,clnt_age,gendr,num_accts,bal,calls_6_mnth,logons_6_mnth
0,836976,6.0,73.0,60.5,U,2.0,45105.30,6.0,9.0
1,2304905,7.0,94.0,58.0,U,2.0,110860.30,6.0,9.0
2,1439522,5.0,64.0,32.0,U,2.0,52467.79,6.0,9.0
3,1562045,16.0,198.0,49.0,M,2.0,67454.65,3.0,6.0
4,5126305,12.0,145.0,33.0,F,2.0,103671.75,0.0,3.0
5,3727881,5.0,71.0,30.5,U,2.0,23915.60,0.0,3.0
6,272934,5.0,66.0,58.5,U,2.0,27021.42,2.0,5.0
7,388801,30.0,361.0,57.5,M,5.0,522498.72,1.0,4.0
8,285619,30.0,369.0,67.5,M,2.0,299388.72,3.0,6.0
9,8198645,15.0,189.0,54.5,F,2.0,382303.83,6.0,9.0


In [4]:
df_clients_profile.shape

(70609, 9)

In [5]:
df_experiment_clients.head(10)

,client_id,Variation
0,9988021,Test
1,8320017,Test
2,4033851,Control
3,1982004,Test
4,9294070,Control
5,9466803,Test
6,3170143,Test
7,6651403,Control
8,5887227,Test
9,2105948,Control


In [6]:
df_experiment_clients.shape

(70609, 2)

In [7]:
df_web_data.head(10)

,client_id,visitor_id,visit_id,process_step,date_time
0,9988021,580560515_7732621733,781255054_21935453173_531117,step_3,2017-04-17 15:27:07
1,9988021,580560515_7732621733,781255054_21935453173_531117,step_2,2017-04-17 15:26:51
2,9988021,580560515_7732621733,781255054_21935453173_531117,step_3,2017-04-17 15:19:22
3,9988021,580560515_7732621733,781255054_21935453173_531117,step_2,2017-04-17 15:19:13
4,9988021,580560515_7732621733,781255054_21935453173_531117,step_3,2017-04-17 15:18:04
5,9988021,580560515_7732621733,781255054_21935453173_531117,step_2,2017-04-17 15:17:15
6,9988021,580560515_7732621733,781255054_21935453173_531117,step_1,2017-04-17 15:17:01
7,9988021,580560515_7732621733,781255054_21935453173_531117,start,2017-04-17 15:16:22
8,8320017,39393514_33118319366,960651974_70596002104_312201,confirm,2017-04-05 13:10:05
9,8320017,39393514_33118319366,960651974_70596002104_312201,step_3,2017-04-05 13:09:43


In [8]:
df_web_data.shape

(755405, 5)

### Exploring numerical and categorical variables

In [9]:
# Retrieving the unique data types present in the dataframe columns
df_clients_profile.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70609 entries, 0 to 70608
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   client_id         70609 non-null  int64  
 1   clnt_tenure_yr    70595 non-null  float64
 2   clnt_tenure_mnth  70595 non-null  float64
 3   clnt_age          70594 non-null  float64
 4   gendr             70595 non-null  object 
 5   num_accts         70595 non-null  float64
 6   bal               70595 non-null  float64
 7   calls_6_mnth      70595 non-null  float64
 8   logons_6_mnth     70595 non-null  float64
dtypes: float64(7), int64(1), object(1)
memory usage: 4.8+ MB


In [10]:
df_experiment_clients.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70609 entries, 0 to 70608
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   client_id  70609 non-null  int64 
 1   Variation  50500 non-null  object
dtypes: int64(1), object(1)
memory usage: 1.1+ MB


In [11]:
df_web_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 755405 entries, 0 to 755404
Data columns (total 5 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   client_id     755405 non-null  int64 
 1   visitor_id    755405 non-null  object
 2   visit_id      755405 non-null  object
 3   process_step  755405 non-null  object
 4   date_time     755405 non-null  object
dtypes: int64(1), object(4)
memory usage: 28.8+ MB


In [12]:
# Extracting column names with numerical data types from the dataframe
numerical_columns_profile = df_clients_profile.select_dtypes(include=[np.number]).columns
print(numerical_columns_profile)

Index(['client_id', 'clnt_tenure_yr', 'clnt_tenure_mnth', 'clnt_age',
       'num_accts', 'bal', 'calls_6_mnth', 'logons_6_mnth'],
      dtype='object')


In [13]:
# Extracting column names with numerical data types from the dataframe
numerical_columns_web = df_web_data.select_dtypes(include=[np.number]).columns
print(numerical_columns_web)

Index(['client_id'], dtype='object')


In [14]:
# Extracting column names with numerical data types from the dataframe
numerical_columns_exp_cli = df_web_data.select_dtypes(include=[np.number]).columns
print(numerical_columns_exp_cli)

Index(['client_id'], dtype='object')


In [15]:
# Separating between discrete and continuous variables, as discrete ones could potentially be treated as categorical.
# Remember to adjust the threshold (in this case, < 20) based on your dataset's specific characteristics and domain knowledge.
potential_categorical_from_numerical_profile = df_clients_profile.select_dtypes("number").loc[:, df_clients_profile.select_dtypes("number").nunique() < 5]
potential_categorical_from_numerical_profile

""
0
1
2
3
4
...
70604
70605
70606
70607


In [16]:
# Separating between discrete and continuous variables, as discrete ones could potentially be treated as categorical.
# Remember to adjust the threshold (in this case, < 20) based on your dataset's specific characteristics and domain knowledge.
potential_categorical_from_numerical_web= df_web_data.select_dtypes("number").loc[:, df_web_data.select_dtypes("number").nunique() < 5]
potential_categorical_from_numerical_web

""
0
1
2
3
4
...
755400
755401
755402
755403


In [17]:
# Separating between discrete and continuous variables, as discrete ones could potentially be treated as categorical.
# Remember to adjust the threshold (in this case, < 20) based on your dataset's specific characteristics and domain knowledge.
potential_categorical_from_numerical_exp_cli= df_experiment_clients.select_dtypes("number").loc[:, df_experiment_clients.select_dtypes("number").nunique() < 5]
potential_categorical_from_numerical_exp_cli

""
0
1
2
3
4
...
70604
70605
70606
70607


In [18]:
# Retrieving column names with object (typically string) data types from the dataframe
df_clients_profile.select_dtypes("object").columns

Index(['gendr'], dtype='object')

In [19]:
# Retrieving column names with object (typically string) data types from the dataframe
df_web_data.select_dtypes("object").columns

Index(['visitor_id', 'visit_id', 'process_step', 'date_time'], dtype='object')

In [20]:
# Retrieving column names with object (typically string) data types from the dataframe
df_experiment_clients.select_dtypes("object").columns

Index(['Variation'], dtype='object')

In [21]:
df_clients_profile.columns

Index(['client_id', 'clnt_tenure_yr', 'clnt_tenure_mnth', 'clnt_age', 'gendr',
       'num_accts', 'bal', 'calls_6_mnth', 'logons_6_mnth'],
      dtype='object')

In [22]:
df_clients_profile['gendr'] = df_clients_profile['gendr'].replace(['U', 'X'], 'Other')

## Drop duplicates values

In [23]:
#clean df_clients_profile
df_clients_profile = print_clean_data(df_clients_profile)

Before
client_id            0
clnt_tenure_yr      14
clnt_tenure_mnth    14
clnt_age            15
gendr               14
num_accts           14
bal                 14
calls_6_mnth        14
logons_6_mnth       14
dtype: int64
------------------------------------------
------------------------------------------
After
client_id           0
clnt_tenure_yr      0
clnt_tenure_mnth    0
clnt_age            0
gendr               0
num_accts           0
bal                 0
calls_6_mnth        0
logons_6_mnth       0
dtype: int64


In [24]:
#clean df_experiment_clients
df_experiment_clients = print_clean_data(df_experiment_clients)

Before
client_id        0
Variation    20109
dtype: int64
------------------------------------------
To many NaN values in column: Variation
------------------------------------------
After
client_id        0
Variation    20109
dtype: int64


In [25]:
#clean df_web_data
df_web_data = print_clean_data(df_web_data)

Before
client_id       0
visitor_id      0
visit_id        0
process_step    0
date_time       0
dtype: int64
------------------------------------------
------------------------------------------
After
client_id       0
visitor_id      0
visit_id        0
process_step    0
date_time       0
dtype: int64


In [26]:
df_web_data.head(20)

,client_id,visitor_id,visit_id,process_step,date_time
0,9988021,580560515_7732621733,781255054_21935453173_531117,step_3,2017-04-17 15:27:07
1,9988021,580560515_7732621733,781255054_21935453173_531117,step_2,2017-04-17 15:26:51
2,9988021,580560515_7732621733,781255054_21935453173_531117,step_3,2017-04-17 15:19:22
3,9988021,580560515_7732621733,781255054_21935453173_531117,step_2,2017-04-17 15:19:13
4,9988021,580560515_7732621733,781255054_21935453173_531117,step_3,2017-04-17 15:18:04
5,9988021,580560515_7732621733,781255054_21935453173_531117,step_2,2017-04-17 15:17:15
6,9988021,580560515_7732621733,781255054_21935453173_531117,step_1,2017-04-17 15:17:01
7,9988021,580560515_7732621733,781255054_21935453173_531117,start,2017-04-17 15:16:22
8,8320017,39393514_33118319366,960651974_70596002104_312201,confirm,2017-04-05 13:10:05
9,8320017,39393514_33118319366,960651974_70596002104_312201,step_3,2017-04-05 13:09:43


# Data Typing/Formatting

In [27]:
df_clients_profile.head(10)

,client_id,clnt_tenure_yr,clnt_tenure_mnth,clnt_age,gendr,num_accts,bal,calls_6_mnth,logons_6_mnth
0,836976,6.0,73.0,60.5,Other,2.0,45105.30,6.0,9.0
1,2304905,7.0,94.0,58.0,Other,2.0,110860.30,6.0,9.0
2,1439522,5.0,64.0,32.0,Other,2.0,52467.79,6.0,9.0
3,1562045,16.0,198.0,49.0,M,2.0,67454.65,3.0,6.0
4,5126305,12.0,145.0,33.0,F,2.0,103671.75,0.0,3.0
5,3727881,5.0,71.0,30.5,Other,2.0,23915.60,0.0,3.0
6,272934,5.0,66.0,58.5,Other,2.0,27021.42,2.0,5.0
7,388801,30.0,361.0,57.5,M,5.0,522498.72,1.0,4.0
8,285619,30.0,369.0,67.5,M,2.0,299388.72,3.0,6.0
9,8198645,15.0,189.0,54.5,F,2.0,382303.83,6.0,9.0


In [28]:
df_clients_profile.dtypes

client_id             int64
clnt_tenure_yr      float64
clnt_tenure_mnth    float64
clnt_age            float64
gendr                object
num_accts           float64
bal                 float64
calls_6_mnth        float64
logons_6_mnth       float64
dtype: object

In [29]:
df_experiment_clients.dtypes

client_id     int64
Variation    object
dtype: object

In [30]:
df_web_data.dtypes

client_id        int64
visitor_id      object
visit_id        object
process_step    object
date_time       object
dtype: object

In [31]:
df_web_data['date_time'] = pd.to_datetime(df_web_data['date_time'], format='%Y-%m-%d %H:%M:%S')

In [32]:
df_web_data.dtypes

client_id                int64
visitor_id              object
visit_id                object
process_step            object
date_time       datetime64[ns]
dtype: object

In [33]:
# Convert specified columns to integer data type
df_clients_profile['clnt_tenure_yr'] = df_clients_profile['clnt_tenure_yr'].astype(int)
df_clients_profile['clnt_tenure_mnth'] = df_clients_profile['clnt_tenure_mnth'].astype(int)
df_clients_profile['clnt_age'] = df_clients_profile['clnt_age'].astype(int)
df_clients_profile['num_accts'] = df_clients_profile['num_accts'].astype(int)
df_clients_profile['calls_6_mnth'] = df_clients_profile['calls_6_mnth'].astype(int)
df_clients_profile['logons_6_mnth'] = df_clients_profile['logons_6_mnth'].astype(int)

In [34]:
df_clients_profile.dtypes

client_id             int64
clnt_tenure_yr        int64
clnt_tenure_mnth      int64
clnt_age              int64
gendr                object
num_accts             int64
bal                 float64
calls_6_mnth          int64
logons_6_mnth         int64
dtype: object

## Create a data frame with the clients that won't participe in the experiment.

In [35]:
#setting aside the clients that wont be part if the experiment
null_df_experiment_clients = df_experiment_clients[df_experiment_clients['Variation'].isnull()]
null_df_experiment_clients

,client_id,Variation
50500,5459747,NaN
50501,8031000,NaN
50502,1847030,NaN
50503,9713157,NaN
50504,7775828,NaN
...,...,...
70604,2443347,NaN
70605,8788427,NaN
70606,266828,NaN
70607,1266421,NaN


## Merge the clients in the web data

### Identify the clients part of the test group

In [36]:
#id of clients that are part of the test
df_experiment_clients_Test = df_experiment_clients[df_experiment_clients['Variation'] == 'Test']
df_experiment_clients_Test.head(10)

,client_id,Variation
0,9988021,Test
1,8320017,Test
3,1982004,Test
5,9466803,Test
6,3170143,Test
8,5887227,Test
10,438567,Test
11,2519971,Test
13,5407849,Test
16,5308474,Test


### df_test: combine visit_id with visitor_id and reoganize the columns.

In [37]:
final_df_Test = pd.merge(df_experiment_clients_Test, df_clients_profile, on='client_id', how='inner')
final_df_Test = pd.merge(df_web_data, final_df_Test, on='client_id', how='inner')
df_test = final_df_Test
df_test.head(15)

,client_id,visitor_id,visit_id,process_step,date_time,Variation,clnt_tenure_yr,clnt_tenure_mnth,clnt_age,gendr,num_accts,bal,calls_6_mnth,logons_6_mnth
0,9988021,580560515_7732621733,781255054_21935453173_531117,step_3,2017-04-17 15:27:07,Test,5,64,79,Other,2,189023.86,1,4
1,9988021,580560515_7732621733,781255054_21935453173_531117,step_2,2017-04-17 15:26:51,Test,5,64,79,Other,2,189023.86,1,4
2,9988021,580560515_7732621733,781255054_21935453173_531117,step_3,2017-04-17 15:19:22,Test,5,64,79,Other,2,189023.86,1,4
3,9988021,580560515_7732621733,781255054_21935453173_531117,step_2,2017-04-17 15:19:13,Test,5,64,79,Other,2,189023.86,1,4
4,9988021,580560515_7732621733,781255054_21935453173_531117,step_3,2017-04-17 15:18:04,Test,5,64,79,Other,2,189023.86,1,4
5,9988021,580560515_7732621733,781255054_21935453173_531117,step_2,2017-04-17 15:17:15,Test,5,64,79,Other,2,189023.86,1,4
6,9988021,580560515_7732621733,781255054_21935453173_531117,step_1,2017-04-17 15:17:01,Test,5,64,79,Other,2,189023.86,1,4
7,9988021,580560515_7732621733,781255054_21935453173_531117,start,2017-04-17 15:16:22,Test,5,64,79,Other,2,189023.86,1,4
8,8320017,39393514_33118319366,960651974_70596002104_312201,confirm,2017-04-05 13:10:05,Test,22,274,34,M,2,36001.90,5,8
9,8320017,39393514_33118319366,960651974_70596002104_312201,step_3,2017-04-05 13:09:43,Test,22,274,34,M,2,36001.90,5,8


### Create the csv file to df_test

In [38]:
df_test.to_csv('../Data/Cleaned_Data/df_test.csv', index=False)

### df_control: combine visit_id with visitor_id and reoganize the columns.

In [39]:
#id of clients that are part of the control group
df_experiment_clients_Control = df_experiment_clients[df_experiment_clients['Variation'] == 'Control']
df_experiment_clients_Control.head(10)

,client_id,Variation
2,4033851,Control
4,9294070,Control
7,6651403,Control
9,2105948,Control
12,9814849,Control
14,7286739,Control
15,4446011,Control
20,4229407,Control
24,2842481,Control
27,1218661,Control


In [40]:
df_experiment_clients_Control = df_experiment_clients[df_experiment_clients['Variation'] == 'Control']
final_df_Control = pd.merge(df_experiment_clients_Control, df_clients_profile, on='client_id', how='inner')
final_df_Control = pd.merge(df_web_data, final_df_Control, on='client_id', how='inner')
df_control = final_df_Control
df_control.head(20)

,client_id,visitor_id,visit_id,process_step,date_time,Variation,clnt_tenure_yr,clnt_tenure_mnth,clnt_age,gendr,num_accts,bal,calls_6_mnth,logons_6_mnth
0,4033851,762728880_76361333336,949661017_22392791362_127391,confirm,2017-04-05 12:29:03,Control,12,149,63,M,2,142642.26,5,8
1,4033851,762728880_76361333336,949661017_22392791362_127391,confirm,2017-04-05 12:29:01,Control,12,149,63,M,2,142642.26,5,8
2,4033851,762728880_76361333336,949661017_22392791362_127391,confirm,2017-04-05 12:28:52,Control,12,149,63,M,2,142642.26,5,8
3,4033851,762728880_76361333336,949661017_22392791362_127391,step_3,2017-04-05 12:26:08,Control,12,149,63,M,2,142642.26,5,8
4,4033851,762728880_76361333336,949661017_22392791362_127391,step_2,2017-04-05 12:24:43,Control,12,149,63,M,2,142642.26,5,8
5,4033851,762728880_76361333336,949661017_22392791362_127391,step_1,2017-04-05 12:24:12,Control,12,149,63,M,2,142642.26,5,8
6,4033851,762728880_76361333336,949661017_22392791362_127391,start,2017-04-05 12:24:04,Control,12,149,63,M,2,142642.26,5,8
7,4033851,762728880_76361333336,949661017_22392791362_127391,step_2,2017-04-05 12:10:26,Control,12,149,63,M,2,142642.26,5,8
8,4033851,762728880_76361333336,949661017_22392791362_127391,step_3,2017-04-05 12:10:26,Control,12,149,63,M,2,142642.26,5,8
9,4033851,762728880_76361333336,949661017_22392791362_127391,step_2,2017-04-05 12:10:25,Control,12,149,63,M,2,142642.26,5,8


### Create the csv file to df_control

In [41]:
df_control.to_csv('../Data/Cleaned_Data/df_control.csv', index=False)

# Pivot and create new columns

### df_test

In [42]:
df_test

,client_id,visitor_id,visit_id,process_step,date_time,Variation,clnt_tenure_yr,clnt_tenure_mnth,clnt_age,gendr,num_accts,bal,calls_6_mnth,logons_6_mnth
0,9988021,580560515_7732621733,781255054_21935453173_531117,step_3,2017-04-17 15:27:07,Test,5,64,79,Other,2,189023.86,1,4
1,9988021,580560515_7732621733,781255054_21935453173_531117,step_2,2017-04-17 15:26:51,Test,5,64,79,Other,2,189023.86,1,4
2,9988021,580560515_7732621733,781255054_21935453173_531117,step_3,2017-04-17 15:19:22,Test,5,64,79,Other,2,189023.86,1,4
3,9988021,580560515_7732621733,781255054_21935453173_531117,step_2,2017-04-17 15:19:13,Test,5,64,79,Other,2,189023.86,1,4
4,9988021,580560515_7732621733,781255054_21935453173_531117,step_3,2017-04-17 15:18:04,Test,5,64,79,Other,2,189023.86,1,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
177782,1574008,117364417_77840596075,528720790_71583064618_169151,confirm,2017-05-06 23:51:05,Test,10,121,55,Other,2,153238.83,3,6
177783,1574008,117364417_77840596075,528720790_71583064618_169151,step_3,2017-05-06 23:50:40,Test,10,121,55,Other,2,153238.83,3,6
177784,1574008,117364417_77840596075,528720790_71583064618_169151,step_2,2017-05-06 23:43:47,Test,10,121,55,Other,2,153238.83,3,6
177785,1574008,117364417_77840596075,528720790_71583064618_169151,step_1,2017-05-06 23:43:34,Test,10,121,55,Other,2,153238.83,3,6


In [43]:
# Assuming df_control is your DataFrame
# 1. Data Preparation:
df_test['date_time'] = pd.to_datetime(df_test['date_time'])
df_test['date'] = df_test['date_time'].dt.date
# Combine visit_id and visitor_id into a new column
df_test['visit_visitor_id'] = df_test['visit_id'].astype(str) + "_" + df_test['visitor_id'].astype(str)

In [44]:
# 2. Calculate time differences per step and make them positive:
# We need to sort the dataframe before calculating the time difference
df_test = df_test.sort_values(by=['visit_id', 'client_id', 'date_time'])

df_test['next_date_time'] = df_test.groupby(['visit_id', 'client_id'])['date_time'].shift(-1)

# Calculate time spent in seconds, but make sure it's always positive
df_test['time_diff_seconds'] = (df_test['next_date_time'] - df_test['date_time']).dt.total_seconds()
df_test['time_diff_minutes'] = df_test['time_diff_seconds'] / 60

In [45]:
# 3. Pivot to Create Step Columns:
df_pivot = df_test.pivot_table(index=['client_id', 'visitor_id', 'visit_id'], columns='process_step', values='time_diff_minutes', fill_value=0, aggfunc='sum')
df_pivot = df_pivot.reset_index()
df_pivot.columns.name = None  # Remove the 'process_step' header

In [46]:
# 4. Rename Columns & Calculate Total Time:
df_pivot = df_pivot.rename(columns={'confirm': 'confirm_time', 'start': 'start_time', 'step_1': 'step_1', 'step_2': 'step_2', 'step_3': 'step_3'})

df_pivot['total_time_visit'] = df_pivot[['start_time', 'step_1', 'step_2', 'step_3', 'confirm_time']].sum(axis=1)

# 5. Merge with Original Data and Select/Reorder Columns:
final_df_t = pd.merge(df_pivot, df_test[['client_id', 'visit_id','Variation', 'clnt_tenure_yr',
       'clnt_tenure_mnth', 'clnt_age', 'gendr', 'num_accts', 'bal',
       'calls_6_mnth', 'logons_6_mnth', 'date', 'process_step']], on=['client_id', 'visit_id'], how='left')

In [47]:
# Get last step per visit_id
final_df_t['last_step'] = final_df_t.groupby('visit_id')['process_step'].transform('last')

# Add new columns bt_1, bt_2, bt_3 for counts of step_1, step_2, step_3
step_counts = df_test.groupby(['visit_visitor_id', 'process_step']).size().unstack(fill_value=0)

# Add bt_1, bt_2, bt_3 columns to final_df
final_df_t['visit_visitor_id'] = final_df_t['visit_id'].astype(str) + "_" + final_df_t['visitor_id'].astype(str)
final_df_t['bt_1'] = final_df_t['visit_visitor_id'].map(step_counts.get('step_1', pd.Series())).fillna(0).astype(int)
final_df_t['bt_2'] = final_df_t['visit_visitor_id'].map(step_counts.get('step_2', pd.Series())).fillna(0).astype(int)
final_df_t['bt_3'] = final_df_t['visit_visitor_id'].map(step_counts.get('step_3', pd.Series())).fillna(0).astype(int)

# Create completion column
final_df_t['completion'] = final_df_t['last_step'].apply(lambda x: 1 if x == 'confirm' else 0)

# Create total_navigation column
final_df_t['total_navigation'] = final_df_t['bt_1'] + final_df_t['bt_2'] + final_df_t['bt_3']

In [48]:
# Drop visit_visitor_id column if no longer needed
final_df_t.drop(['visit_id', 'visitor_id','process_step', 'confirm_time'], axis=1, inplace=True)
final_df_t.drop_duplicates(subset='visit_visitor_id', keep='first', inplace=True)
# Reorder columns to place total_navigation after bt_3 and completed at the end
df_test_final = final_df_t.reindex(columns=[
    'client_id', 'visit_visitor_id', 'start_time', 'step_1', 'step_2', 'step_3', 
    'date', 'bt_1', 'bt_2', 'bt_3', 'total_navigation', 'last_step', 'completion',
    'total_time_visit', 'Variation', 'clnt_tenure_yr', 'clnt_tenure_mnth', 
    'clnt_age', 'gendr', 'num_accts', 'bal', 'calls_6_mnth', 'logons_6_mnth'
])

In [49]:
df_test_final.head(10)

,client_id,visit_visitor_id,start_time,step_1,step_2,step_3,date,bt_1,bt_2,bt_3,total_navigation,last_step,completion,total_time_visit,Variation,clnt_tenure_yr,clnt_tenure_mnth,clnt_age,gendr,num_accts,bal,calls_6_mnth,logons_6_mnth
0,555,637149525_38041617439_716659_402506806_5608737...,0.116667,0.533333,1.650000,0.333333,2017-04-15,1,1,1,3,confirm,1,2.633333,Test,3,46,29,Other,2,25454.66,2,6
5,647,40369564_40101682850_311847_66758770_53988066587,0.116667,0.300000,3.150000,2.716667,2017-04-12,1,1,1,3,confirm,1,6.283333,Test,12,151,57,M,2,30525.80,0,4
10,934,7076463_57954418406_971348_810392784_45004760546,2.366667,0.000000,0.000000,0.000000,2017-04-18,0,0,0,0,start,0,2.366667,Test,9,109,51,F,2,32522.88,0,3
14,1336,583743392_96265099036_939815_920624746_3260333...,0.416667,0.433333,0.850000,1.816667,2017-05-08,1,1,1,3,confirm,1,3.516667,Test,48,576,42,M,4,130537.18,6,9
19,1336,614001770_19101025926_112779_920624746_3260333...,0.000000,0.000000,0.000000,0.000000,2017-05-08,0,0,0,0,confirm,1,1.366667,Test,48,576,42,M,4,130537.18,6,9
21,1346,27144337_83739845380_214282_123474046_4204671056,2.650000,3.366667,0.816667,1.416667,2017-06-06,2,1,2,5,step_3,0,8.250000,Test,14,177,46,F,2,822512.91,3,6
31,1516,255400977_38039535960_779641_182314299_6316858...,0.933333,0.583333,10.116667,4.266667,2017-04-06,1,1,1,3,confirm,1,15.900000,Test,12,150,58,F,2,25408.39,5,8
36,1516,481123290_95510977345_707323_182314299_6316858...,1.333333,1.666667,1.083333,0.000000,2017-04-05,3,3,0,6,step_1,0,4.083333,Test,12,150,58,F,2,25408.39,5,8
47,1643,753205700_16851596206_134483_121807985_2052456...,1.000000,0.266667,9.700000,0.500000,2017-06-10,1,1,1,3,confirm,1,11.466667,Test,19,229,55,M,2,81731.39,0,3
52,1643,379946188_1773022140_107963_910842861_51062379179,0.000000,0.000000,0.000000,0.000000,2017-04-15,0,0,0,0,start,0,0.000000,Test,19,229,55,M,2,81731.39,0,3


In [50]:
df_test_final.info()

<class 'pandas.core.frame.DataFrame'>
Index: 37122 entries, 0 to 177785
Data columns (total 23 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   client_id         37122 non-null  int64  
 1   visit_visitor_id  37122 non-null  object 
 2   start_time        37122 non-null  float64
 3   step_1            37122 non-null  float64
 4   step_2            37122 non-null  float64
 5   step_3            37122 non-null  float64
 6   date              37122 non-null  object 
 7   bt_1              37122 non-null  int64  
 8   bt_2              37122 non-null  int64  
 9   bt_3              37122 non-null  int64  
 10  total_navigation  37122 non-null  int64  
 11  last_step         37122 non-null  object 
 12  completion        37122 non-null  int64  
 13  total_time_visit  37122 non-null  float64
 14  Variation         37122 non-null  object 
 15  clnt_tenure_yr    37122 non-null  int64  
 16  clnt_tenure_mnth  37122 non-null  int64  
 1

In [72]:
df_test_final = convert_data_types(df_test_final)

In [52]:
df_test_final.dtypes

client_id                   object
visit_visitor_id            object
start_time                 float64
step_1                     float64
step_2                     float64
step_3                     float64
date                datetime64[ns]
bt_1                         int64
bt_2                         int64
bt_3                         int64
total_navigation             int64
last_step                   object
completion                    bool
total_time_visit           float64
Variation                   object
clnt_tenure_yr               int64
clnt_tenure_mnth             int64
clnt_age                     int64
gendr                       object
num_accts                    int64
bal                        float64
calls_6_mnth                 int64
logons_6_mnth                int64
dtype: object

### Create the csv file to df_test_final

In [53]:
df_test_final.to_csv('../Data/Cleaned_Data/df_test_final.csv', index=False)

### df_control

In [54]:
numerical_cols = ['total_navigation']
correlations = df_test_final[numerical_cols + ['completion']].corr()['completion'].drop('completion')
correlations

total_navigation    0.237913
Name: completion, dtype: float64

In [55]:
df_control.head(10)

,client_id,visitor_id,visit_id,process_step,date_time,Variation,clnt_tenure_yr,clnt_tenure_mnth,clnt_age,gendr,num_accts,bal,calls_6_mnth,logons_6_mnth
0,4033851,762728880_76361333336,949661017_22392791362_127391,confirm,2017-04-05 12:29:03,Control,12,149,63,M,2,142642.26,5,8
1,4033851,762728880_76361333336,949661017_22392791362_127391,confirm,2017-04-05 12:29:01,Control,12,149,63,M,2,142642.26,5,8
2,4033851,762728880_76361333336,949661017_22392791362_127391,confirm,2017-04-05 12:28:52,Control,12,149,63,M,2,142642.26,5,8
3,4033851,762728880_76361333336,949661017_22392791362_127391,step_3,2017-04-05 12:26:08,Control,12,149,63,M,2,142642.26,5,8
4,4033851,762728880_76361333336,949661017_22392791362_127391,step_2,2017-04-05 12:24:43,Control,12,149,63,M,2,142642.26,5,8
5,4033851,762728880_76361333336,949661017_22392791362_127391,step_1,2017-04-05 12:24:12,Control,12,149,63,M,2,142642.26,5,8
6,4033851,762728880_76361333336,949661017_22392791362_127391,start,2017-04-05 12:24:04,Control,12,149,63,M,2,142642.26,5,8
7,4033851,762728880_76361333336,949661017_22392791362_127391,step_2,2017-04-05 12:10:26,Control,12,149,63,M,2,142642.26,5,8
8,4033851,762728880_76361333336,949661017_22392791362_127391,step_3,2017-04-05 12:10:26,Control,12,149,63,M,2,142642.26,5,8
9,4033851,762728880_76361333336,949661017_22392791362_127391,step_2,2017-04-05 12:10:25,Control,12,149,63,M,2,142642.26,5,8


In [56]:
# Assuming df_control is your DataFrame
# 1. Data Preparation:
df_control['date_time'] = pd.to_datetime(df_control['date_time'])
df_control['date'] = df_control['date_time'].dt.date
# Combine visit_id and visitor_id into a new column
df_control['visit_visitor_id'] = df_control['visit_id'].astype(str) + "_" + df_control['visitor_id'].astype(str)

In [57]:
# 2. Calculate time differences per step and make them positive:
# We need to sort the dataframe before calculating the time difference
df_control = df_control.sort_values(by=['visit_id', 'client_id', 'date_time'])

df_control['next_date_time'] = df_control.groupby(['visit_id', 'client_id'])['date_time'].shift(-1)

# Calculate time spent in seconds, but make sure it's always positive
df_control['time_diff_seconds'] = (df_control['next_date_time'] - df_control['date_time']).dt.total_seconds()
df_control['time_diff_minutes'] = df_control['time_diff_seconds'] / 60

In [58]:
# 3. Pivot to Create Step Columns:
df_pivot = df_control.pivot_table(index=['client_id', 'visitor_id', 'visit_id'], columns='process_step', values='time_diff_minutes', fill_value=0, aggfunc='sum')
df_pivot = df_pivot.reset_index()
df_pivot.columns.name = None  # Remove the 'process_step' header

In [59]:
# 4. Rename Columns & Calculate Total Time:
df_pivot = df_pivot.rename(columns={'confirm': 'confirm_time', 'start': 'start_time', 'step_1': 'step_1', 'step_2': 'step_2', 'step_3': 'step_3'})

df_pivot['total_time_visit'] = df_pivot[['start_time', 'step_1', 'step_2', 'step_3', 'confirm_time']].sum(axis=1)

# 5. Merge with Original Data and Select/Reorder Columns:
final_df_c = pd.merge(df_pivot, df_control[['client_id', 'visit_id','Variation', 'clnt_tenure_yr',
       'clnt_tenure_mnth', 'clnt_age', 'gendr', 'num_accts', 'bal',
       'calls_6_mnth', 'logons_6_mnth', 'date', 'process_step']], on=['client_id', 'visit_id'], how='left')

In [60]:
# Get last step per visit_id
final_df_c['last_step'] = final_df_c.groupby('visit_id')['process_step'].transform('last')

# Add new columns bt_1, bt_2, bt_3 for counts of step_1, step_2, step_3
step_counts = df_control.groupby(['visit_visitor_id', 'process_step']).size().unstack(fill_value=0)

# Add bt_1, bt_2, bt_3 columns to final_df
final_df_c['visit_visitor_id'] = final_df_c['visit_id'].astype(str) + "_" + final_df_c['visitor_id'].astype(str)
final_df_c['bt_1'] = final_df_c['visit_visitor_id'].map(step_counts.get('step_1', pd.Series())).fillna(0).astype(int)
final_df_c['bt_2'] = final_df_c['visit_visitor_id'].map(step_counts.get('step_2', pd.Series())).fillna(0).astype(int)
final_df_c['bt_3'] = final_df_c['visit_visitor_id'].map(step_counts.get('step_3', pd.Series())).fillna(0).astype(int)

# Create completion column
final_df_c['completion'] = final_df_c['last_step'].apply(lambda x: 1 if x == 'confirm' else 0)

# Create total_navigation column
final_df_c['total_navigation'] = final_df_c['bt_1'] + final_df_t['bt_2'] + final_df_t['bt_3']

In [61]:
# Drop visit_visitor_id column if no longer needed
final_df_c.drop(['visit_id', 'visitor_id', 'process_step', 'confirm_time'], axis=1, inplace=True)
final_df_c.drop_duplicates(subset='visit_visitor_id', keep='first', inplace=True)

In [62]:
# Reorder columns to place total_navigation after bt_3 and completed at the end
df_control_final = final_df_c.reindex(columns=[
    'client_id', 'visit_visitor_id', 'start_time', 'step_1', 'step_2', 'step_3', 
    'date', 'bt_1', 'bt_2', 'bt_3', 'total_navigation', 'last_step', 'completion',
    'total_time_visit', 'Variation', 'clnt_tenure_yr', 'clnt_tenure_mnth', 
    'clnt_age', 'gendr', 'num_accts', 'bal', 'calls_6_mnth', 'logons_6_mnth'
])

In [63]:
df_control_final = convert_data_types(df_control_final)

In [64]:
df_control_final.dtypes

client_id                   object
visit_visitor_id            object
start_time                 float64
step_1                     float64
step_2                     float64
step_3                     float64
date                datetime64[ns]
bt_1                         int64
bt_2                         int64
bt_3                         int64
total_navigation             int64
last_step                   object
completion                    bool
total_time_visit           float64
Variation                   object
clnt_tenure_yr               int64
clnt_tenure_mnth             int64
clnt_age                     int64
gendr                       object
num_accts                    int64
bal                        float64
calls_6_mnth                 int64
logons_6_mnth                int64
dtype: object

### Create the csv file to df_test_final

In [65]:
df_control_final.to_csv('../Data/Cleaned_Data/df_control_final.csv', index=False)

# New Data Frames

## df_control_final

In [66]:
df_control_final.head(10)

,client_id,visit_visitor_id,start_time,step_1,step_2,step_3,date,bt_1,bt_2,bt_3,total_navigation,last_step,completion,total_time_visit,Variation,clnt_tenure_yr,clnt_tenure_mnth,clnt_age,gendr,num_accts,bal,calls_6_mnth,logons_6_mnth
0,1028,557292053_87239438319_391157_42237450_62128060588,0.82,1.57,4.73,1.85,2017-04-08,5,2,1,7,step_1,False,8.97,Control,12,145,36,M,3,103520.22,1,4
9,1104,543158812_46395476577_767725_194240915_1815800...,0.00,0.00,0.00,0.00,2017-06-12,0,0,0,0,start,False,0.00,Control,5,66,48,Other,3,154643.94,6,9
10,1104,643221571_99977972121_69283_194240915_18158000533,0.00,0.00,0.00,0.00,2017-06-20,0,0,0,0,start,False,0.00,Control,5,66,48,Other,3,154643.94,6,9
11,1186,507052512_11309370126_442139_446844663_3161510...,0.00,0.00,0.00,0.00,2017-04-08,0,0,0,0,start,False,0.00,Control,8,99,22,Other,2,31662.52,0,3
12,1186,795373564_99931517312_810896_446844663_3161510...,0.18,0.18,0.00,0.00,2017-04-08,1,1,0,0,step_2,False,0.37,Control,8,99,22,Other,2,31662.52,0,3
15,1195,393817425_39015278493_996341_766842522_6999255...,0.55,1.63,0.52,1.38,2017-04-05,1,1,1,0,confirm,True,4.08,Control,21,262,54,M,2,28457.96,2,5
20,1197,71862471_21202285428_848395_753759429_54481946928,0.07,0.10,0.95,0.47,2017-04-01,1,2,2,0,confirm,True,1.58,Control,7,85,28,Other,5,177062.14,5,8
27,1368,784065271_45379483290_309335_366307863_1901466...,0.00,0.00,0.00,0.00,2017-04-09,0,0,0,0,start,False,0.00,Control,5,60,45,F,3,61956.25,6,9
28,2439,848231744_22569944243_37711_607208067_70160939111,0.00,0.00,0.00,0.00,2017-03-23,0,0,0,0,start,False,0.00,Control,11,142,56,M,2,504142.55,0,3
29,2581,182925466_27021409208_83502_770616558_80928163524,0.00,0.00,0.00,0.00,2017-04-09,0,0,0,0,start,False,0.00,Control,4,58,22,M,2,24976.79,1,4


## df_test_final

In [67]:
df_test_final.head(10)

,client_id,visit_visitor_id,start_time,step_1,step_2,step_3,date,bt_1,bt_2,bt_3,total_navigation,last_step,completion,total_time_visit,Variation,clnt_tenure_yr,clnt_tenure_mnth,clnt_age,gendr,num_accts,bal,calls_6_mnth,logons_6_mnth
0,555,637149525_38041617439_716659_402506806_5608737...,0.12,0.53,1.65,0.33,2017-04-15,1,1,1,3,confirm,True,2.63,Test,3,46,29,Other,2,25454.66,2,6
5,647,40369564_40101682850_311847_66758770_53988066587,0.12,0.30,3.15,2.72,2017-04-12,1,1,1,3,confirm,True,6.28,Test,12,151,57,M,2,30525.80,0,4
10,934,7076463_57954418406_971348_810392784_45004760546,2.37,0.00,0.00,0.00,2017-04-18,0,0,0,0,start,False,2.37,Test,9,109,51,F,2,32522.88,0,3
14,1336,583743392_96265099036_939815_920624746_3260333...,0.42,0.43,0.85,1.82,2017-05-08,1,1,1,3,confirm,True,3.52,Test,48,576,42,M,4,130537.18,6,9
19,1336,614001770_19101025926_112779_920624746_3260333...,0.00,0.00,0.00,0.00,2017-05-08,0,0,0,0,confirm,True,1.37,Test,48,576,42,M,4,130537.18,6,9
21,1346,27144337_83739845380_214282_123474046_4204671056,2.65,3.37,0.82,1.42,2017-06-06,2,1,2,5,step_3,False,8.25,Test,14,177,46,F,2,822512.91,3,6
31,1516,255400977_38039535960_779641_182314299_6316858...,0.93,0.58,10.12,4.27,2017-04-06,1,1,1,3,confirm,True,15.90,Test,12,150,58,F,2,25408.39,5,8
36,1516,481123290_95510977345_707323_182314299_6316858...,1.33,1.67,1.08,0.00,2017-04-05,3,3,0,6,step_1,False,4.08,Test,12,150,58,F,2,25408.39,5,8
47,1643,753205700_16851596206_134483_121807985_2052456...,1.00,0.27,9.70,0.50,2017-06-10,1,1,1,3,confirm,True,11.47,Test,19,229,55,M,2,81731.39,0,3
52,1643,379946188_1773022140_107963_910842861_51062379179,0.00,0.00,0.00,0.00,2017-04-15,0,0,0,0,start,False,0.00,Test,19,229,55,M,2,81731.39,0,3


### Create the csv file to combine_df

In [68]:
combined_df = pd.concat([df_test_final, df_control_final], ignore_index=True)

In [73]:
combined_df = convert_data_types(combined_df)

In [69]:
combined_df.to_csv('../Data/Cleaned_Data/combined_df.csv', index=False)

In [70]:
combined_df.sample(5)

,client_id,visit_visitor_id,start_time,step_1,step_2,step_3,date,bt_1,bt_2,bt_3,total_navigation,last_step,completion,total_time_visit,Variation,clnt_tenure_yr,clnt_tenure_mnth,clnt_age,gendr,num_accts,bal,calls_6_mnth,logons_6_mnth
19473,5251384,506615726_43478466512_142378_428108418_8013646...,0.93,0.00,0.00,0.00,2017-04-18,0,0,0,0,start,False,0.93,Test,23,276,68,F,2,255382.63,4,7
15127,4075832,557615009_87114638538_404641_935053144_6465287...,0.37,0.40,1.20,0.63,2017-04-15,1,1,1,3,confirm,True,2.60,Test,8,98,28,Other,2,56645.58,3,6
16700,4514170,618330524_78263712932_449767_769241800_9063346...,0.92,0.57,1.03,3.07,2017-04-19,1,1,1,3,confirm,True,5.58,Test,16,195,69,F,2,116889.22,3,6
13761,3725702,525947299_79403078238_804927_771043347_7619882...,0.07,0.12,1.12,0.33,2017-04-15,1,1,1,3,confirm,True,1.63,Test,6,72,58,Other,2,63541.45,6,9
31751,8540292,266689703_99113418224_874865_542228910_9692168...,0.72,0.85,1.28,1.80,2017-04-06,1,1,1,3,confirm,True,4.65,Test,14,170,60,M,2,507257.91,3,6


In [74]:
combined_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 69303 entries, 0 to 69302
Data columns (total 23 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   client_id         69303 non-null  object        
 1   visit_visitor_id  69303 non-null  object        
 2   start_time        69303 non-null  float64       
 3   step_1            69303 non-null  float64       
 4   step_2            69303 non-null  float64       
 5   step_3            69303 non-null  float64       
 6   date              69303 non-null  datetime64[ns]
 7   bt_1              69303 non-null  int64         
 8   bt_2              69303 non-null  int64         
 9   bt_3              69303 non-null  int64         
 10  total_navigation  69303 non-null  int64         
 11  last_step         69303 non-null  object        
 12  completion        69303 non-null  bool          
 13  total_time_visit  69303 non-null  float64       
 14  Variation         6930